# ***Priv Esc Runbook***

## Setup

In [ ]:
# install packages
print("Installing Necesssary Packages, please wait.")
!{sys.executable} -m pip install kqlmagic --no-cache-dir --upgrade
!{sys.executable} -m pip install msticpy
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install datetime
!{sys.executable} -m pip install multihash
!{sys.executable} -m pip install os
!{sys.executable} -m pip install seaborn
print("Package installation complete.")

In [ ]:
#import packages
print("Inporting Necesssary Packages, please wait.")
import sys
import Kqlmagic
import msticpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os
import seaborn as sns
print("Package import complete.")
%reload_ext Kqlmagic

In [ ]:
#Adding IOCs as Global Variables to be used across functions
badguys = ("badguyusername", "badguy2")
compromised_accounts = ("admin1, engineer2")
start_time = "2022-02-23 00:00:00"
end_time = "2022-02-28 00:00:00"
privesc_tools = ("winPEAs", "adPEAs", "mimikatz", "powerup", "sherlock", "seatbelt". "jaws-enum", "wes.py", "privesccheck")
tool_used = ()
attribution = ()

In [ ]:
#add all necessary kusto connections
%kql kusto://code().cluster('WEC').database('logs')
%kql kusto://code().cluster('AV').database('logs')

## Queries

In [ ]:
initial_wec == '''
%%kql logs@WEC
    let badguys = usernames_list;
    let privesc_tools = privesc_tools;
    let start_time = start_time
    let end_time = end_time
    cluster(WEC).database(logs).SecurityLog
    | where ingestion_time() between(start_time..end_time)
    | where EventID == 4688
    | where EventData has_any usernames_list
    | where EventData.CommandLine has_any privesc_tools
    | extend SubjectUserName = tostring(parse_json(EventData).SubjectUserName),
            TargetUserName = tostring(parse_json(EventData).TargetUserName),
            CommandLine = tostring(parse_json(EventData).CommandLine),
            ParentProcessName = tostring(parse_json(EventData).ParentProcessName),
            NewProcessName = tostring(parse_json(EventData).NewProcessName)
    | project TimeGenerated, Computer, SubjectUserName, TargetUserName, ParentProcessName, NewProcessName, CommandLine
    | limit 1000
'''
InitialWECLogs = _kql_raw_result_.to_dataframe().sort_values('TimeGenerated', ascending = False)
InitialWECLogs.head(50)

In [ ]:
# the %1937 tokenelevation type indicates a user running with admin or equivalent permissions, 
# similar to executing with runas
token_elevation = '''
%%kql logs@WEC
    let start_time = start_time
    let end_time = end_time
    cluster(WEC).database(logs).SecurityLog
    | where ingestion_time() between(start_time..end_time)
    | where EventID == 4688
    | where Computer contains "JMP"
    | where EventData.TokenElevation.Type contains "1937"
    | sort by TimeGenerated desc
    | limit 1000
'''
Token_Elevation = _kql_raw_result_.to_dataframe().sort_values('TimeGenerated', ascending = False)
Token_Elevation.head(50)

In [ ]:
# suspicious rundll32, this is a technique of dll hijacking
rundll_hijacking = '''
%%kql logs@WEC
    let start_time = start_time
    let end_time = end_time
    cluster(WEC).database(logs).SecurityLog
    | where ingestion_time() between(start_time..end_time)
    | where EventID == 4688
    | where EventData.CommandLine contains "rundll32"
    | where EventData.NewProcessName !contains "system32"
    | sort by TimeGenerated desc
    | limit 1000
'''
rundll_results = _kql_raw_result_.to_dataframe().sort_values('TimeGenerated', ascending = False)
rundll_results.head(50)

In [ ]:
# querying for possible pass the hash techniques - still working on this
pass_the_hash = '''
%%kql logs@WEC
    let start_time = start_time
    let end_time = end_time
    cluster(WEC).database(logs).SecurityLog
    | where TimeCreated between(start_time..end_time)
    | where EventID == 4624 and EventData.LogonType == 9
    | where isnotempty(EventData.TargetOutBoundUserName) and EventData.TargetOutBoundUserName != "-"
    | project TimeCreated, Computer, EventData.TargetOutBoundUserName, EventData.LogonType, EventData.IpAddress EventData.LogonGuid 
    | join (SecurityLog
           | where TimeCreated between(start_time..end_time)
           | where EventID == 4648
           | where EventData.TargetServerName != "-" and EventData.TargetServerName != "localhost"
           | project TimeCreated, EventData.TargetServerName, Computer, EventData.TargetUserName, EventData.TargetInfo,
            EventData.LogonGiud
           ) on EventData.LogonGuid
    | sort by TimeCreated desc
'''
pth_results = _kql_raw_result_.to_dataframe().sort_values('TimeGenerated', ascending = False)
pth_results.head(50)

In [ ]:
# user logins
user_logins = '''
%%kql logs@WEC
let start_time = $start_time;
let end_time = $end_time;
let users = $compromised_accounts;
cluster(WEC).database(logs).SecurityLog
| where TimeCreated between(start_time..end_time)
| where EventId == 4624
| where EventData.TargetUserName in~(users)
| extend TargetUserName = tostring(EventData.TargetUserName),
    TargetUserSid = EventData.TargetUserSid,
    IpAddress = tostring(EventData.IpAddress),
    LogonType = tostring(EventData.LogonType)
| where TargetUserSid !~ 'S-1-5-18'
| summarize Count=count() by Computer,TargetUserName, IpAddress, LogonType"
'''

user_login_results = _kql_raw_result_.to_dataframe().sort_values('TimeGenerated', ascending = False)

In [ ]:
#logins by machine
computer_logins = '''
%%kql logs@WEC
let start_time = $start_time;
let end_time = $end_time;
let computers = $compromised_accounts;
cluster(WEC).database(logs).SecurityLog
| where TimeCreated between(start_time..end_time)
| where EventId == 4624
| where Computer in~(computers)
| extend TargetUserName = tostring(EventData.TargetUserName),
    TargetUserSid = EventData.TargetUserSid,
    IpAddress = tostring(EventData.IpAddress),
    LogonType = tostringEventData.LogonType)
| where TargetUserSid !~ 'S-1-5-18'
| summarize Count=count() by TargetUserName, Computer, IpAddress, LogonType"
'''

computer_login_results = _kql_raw_result_.to_dataframe().sort_values('TimeGenerated', ascending = False)

## Visualizations

In [ ]:
sns.barplot(x = 'TargetUserName',
            y = 'Count',
            hue = 'Computer',
            data = user_login_results)

plt.show()

In [ ]:
sns.barplot(x = "Computer",
            y = "Count",
            hue = "TargetUserName",
            data = computer_login_results)

plt.show()        

## Analytics

In [ ]:
#all unique commandline arguments
for cmd in InitialWECLogs['CommandLine'].unique():
    print(cmd)
    
#find any powershell encoded commands
encoded_filter = InitialWECLogs['CommandLine'].str.find("encodedcommand")
encoded_filter

#number of processes spawned by each parent
#this is something we could baseline as well - notate the values over the course of a normal week for example and
proc_relationship = InitialWECLogs.groupby(['ParentProcessName']).size()
proc_relationship